# How to connect a SageMaker notebook to Snowflake using Python

In this notebook, I will walk through the steps to connect to a Snowflake data warehouse and pull data and tables to a dataframe on the Sagemaker notebook. This is largely explained in the post from Robert Fehrmann here: https://www.snowflake.com/blog/connecting-a-jupyter-notebook-to-snowflake-through-python-part-3/.

The connection is achieved using the Snowflake Connector for Python, which is a pure Python package that has no dependencies on JDBC or ODBC. More can be read here: https://docs.snowflake.com/en/user-guide/python-connector.html.

First check if the necessary dependencies are installed, and if not (or there is an older version), de-install and reinstall the required version.

In [ ]:
%%bash
CFFI_VERSION=$(pip list 2>/dev/null | grep cffi )
echo $CFFI_VERSION
if [[ "$CFFI_VERSION" == "cffi (1.10.0)" ]]
then 
   pip uninstall --yes cffi
fi
yum_log=$(sudo yum install -y libffi-devel openssl-devel)
pip_log=$(pip install --upgrade snowflake-connector-python)  
if [[ "$CFFI_VERSION" == "cffi (1.10.0)" ]]
then 
   echo "configuration has changed; restart notebook"
fi

Install the Pandas-compatible version of the Snowflake Connector for Python. While not necessary (and perfectly fine to just use the snowflake connector imported below), it allows you to use some Pandas-oriented APIs with the Snowflake connector (like `fetch_pandas_all()` below).

You may have to update the snowflake connector for some of the APIs that work with pandas dataframes to work properly. Use the following line of code to do so:

`!pip install snowflake-connector-python[pandas]`

In [ ]:
import snowflake.connector

# Connecting to Snowflake using the default authenticator
# You will have to enter appropriate strings for the required connector parameters
'''
param_values = {
    '/SNOWFLAKE/USER_ID': '',
    '/SNOWFLAKE/PASSWORD': '',
    '/SNOWFLAKE/ACCOUNT_ID': '',
    '/SNOWFLAKE/WAREHOUSE': '',
    '/SNOWFLAKE/DATABASE': '',
    '/SNOWFLAKE/SCHEMA': ''
}
'''

ctx = snowflake.connector.connect(
  user=param_values['/SNOWFLAKE/USER_ID'],
  password=param_values['/SNOWFLAKE/PASSWORD'],
  account=param_values['/SNOWFLAKE/ACCOUNT_ID'],
  warehouse=param_values['/SNOWFLAKE/WAREHOUSE'],
  database=param_values['/SNOWFLAKE/DATABASE'],
  schema=param_values['/SNOWFLAKE/SCHEMA'])

Note: it is actually a best practice to use a secure key/value management system, like AWS Systems Manger Paramter Store (SSM) to store and manage your credentials, as opposed to hard-coding them. You can read more about AWS SSM here: https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html.

In [ ]:
# Create a cursor object.
cur = ctx.cursor()

In [ ]:
# Execute a statement that will generate a result set.
sql = "select * from MGML_AWS_SOURCE"
cur.execute(sql)

If you opted not to install the newer version of the snowflake python connector you can use the following line of code to assign the executed sql command: `allrows = cur.execute(sql)`

In [ ]:
import pandas as pd
# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.

df = cur.fetch_pandas_all()

If you opted not to install the newer version of the snowflake python connector you can use the following line of code to assign the executed sql command to a pandas dataframe: `df = pd.DataFrame(allrows)`.

In [ ]:
df.head()

Next, we manipulate a slice of the dataframe and write it back to the table in Snowflake. This could be your results at the end of ML pipeline, Step Functions, etc.

Or, if you write large outputs to an S3 bucket for staging and/or cost, you can create and execute a connection with the snowflake connector object and move those files directly from S3 to Snowflake. There is a sample program demonstrating this in the Snowflake documentation linked above. I will reporoduce the link here: https://docs.snowflake.com/en/user-guide/python-connector-example.html

In [ ]:
from snowflake.connector.pandas_tools import write_pandas
import random
from datetime import datetime

random.seed(datetime.now())
temp = random.randint(0,100)

df.loc[0,('X0')] = temp

df.head()

Notice we have changed the value of the cell under column 'X0' in the 0th index

In [ ]:
# Write the data from the DataFrame to the table named "MGML_AWS_SOURCE".
success, nchunks, nrows, _ = write_pandas(ctx, df, 'MGML_AWS_SOURCE')
print(success)

You can double check the success of the write operation by running a SELECT * on your table in Snowflake